In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = pd.read_csv("data/data_500k.csv")
data.head()

C:\Users\Hernando\AppData\Local\Temp\ipykernel_12968\610779890.py:1: DtypeWarning: Columns (22,56,58,86,93,117,144,146,148,150,159,164,169,171,178,192,193,205,214,215,241,293,297) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/data_500k.csv")


,RECORD_ID,ADD_TYPE,AFAMPROFLS,AGE,AI_COUNTY_NAME,AIRCOND,APP_CHILD,APP_MENBIG,APP_TODDLR,APP_WOMEN,...,VTR_PRI16,VTR_PRI17,VTR_PRI18,VTR_PRI19,VTR_PRI20,VTR_PRI21,VTR_PRI22,WORKWOMAN,YEARBUILT,ZIP
0,403390,S,,21.0,Fairbanks North Star,,,,,,...,,,,,,,,,NaN,99705
1,62285,H,,NaN,Anchorage,,,,,,...,,,,,,,,,NaN,99506
2,331355,,,91.0,Kenai Peninsula,,,,,,...,,,,,,,,,NaN,99603
3,206320,H,,65.0,Anchorage,,,,,,...,,,,,,,Y,,,99567
4,188078,S,,76.0,Juneau,,,,,,...,Y,,Y,,,,Y,Y,1985,99801


In [4]:
data_str = "VTR_GEN00	VTR_GEN01	VTR_GEN02	VTR_GEN03	VTR_GEN04	VTR_GEN05	VTR_GEN06	VTR_GEN07	VTR_GEN08	VTR_GEN09	VTR_GEN10	VTR_GEN11	VTR_GEN12	VTR_GEN13	VTR_GEN14	VTR_GEN15	VTR_GEN16	VTR_GEN17	VTR_GEN18	VTR_GEN19	VTR_GEN20	VTR_GEN21	VTR_GEN22	VTR_OTH00	VTR_OTH01	VTR_OTH02	VTR_OTH03	VTR_OTH04	VTR_OTH05	VTR_OTH06	VTR_OTH07	VTR_OTH08	VTR_OTH09	VTR_OTH10	VTR_OTH11	VTR_OTH12	VTR_OTH13	VTR_OTH14	VTR_OTH15	VTR_OTH16	VTR_OTH17	VTR_OTH18	VTR_OTH19	VTR_OTH20	VTR_OTH21	VTR_OTH22	VTR_PPP00	VTR_PPP04	VTR_PPP08	VTR_PPP12	VTR_PPP16	VTR_PRI00	VTR_PRI01	VTR_PRI02	VTR_PRI03	VTR_PRI04	VTR_PRI05	VTR_PRI06	VTR_PRI07	VTR_PRI08	VTR_PRI09	VTR_PRI10	VTR_PRI11	VTR_PRI12	VTR_PRI13	VTR_PRI14	VTR_PRI15	VTR_PRI16	VTR_PRI17	VTR_PRI18	VTR_PRI19	VTR_PRI20	VTR_PRI21	VTR_PRI22"

# Convert the string into a list by splitting on tabs
data_list = data_str.split("\t")

string= """PARTY_CODE
PARTY_MIX
PRFL_2NDAMEND
PRFL_ACTIVE_MIL
PRFL_AMZN_PRIME
PRFL_ANML_RIGHTS
PRFL_BIDEN_SUPPORT
PRFL_BLM_SUPPORT
PRFL_BORDER_SECURITY
PRFL_CHOICELIFE
PRFL_CLINTON_SUPPORT
PRFL_CONSERVATIVE_NEWS
PRFL_EDUCATION
PRFL_ENVIRONMENT
PRFL_EVANGELICAL
PRFL_FENCE_SITTER
PRFL_GUN_CONTROL
PRFL_HEALTHCARE
PRFL_HEALTHCARE_REFORM
PRFL_IMMIGRATION_REFORM
PRFL_INFLUENCER
PRFL_INSURANCE
PRFL_LABOR
PRFL_LGBT_SUPPORT
PRFL_LIBERAL_NEWS
PRFL_MARIJUANA_REFORM
PRFL_MARRIAGE_EQUALITY
PRFL_METOO_SUPPORT
PRFL_MIL_SUPPORT
PRFL_MINWAGE
PRFL_OBAMA
PRFL_PERSUADABLE_VOTER
PRFL_POLITICAL_IDEOLOGY
PRFL_SANDERS_SUPPORT
PRFL_TAXES
PRFL_TEACHERS_UNION
PRFL_TEAPARTY
PRFL_TRUMP_SUPPORT
PRFL_VETERAN"""

data_list += string.split("\n")

data.drop(columns=data_list, inplace=True)


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

# Filter the dataset for the rows with 'D' or 'R' values in the 'VTR_PPP20' column
filtered_data = data[data['VTR_PPP20'].isin(['D', 'R'])]

# Split the data into training (80%) and testing (20%) sets
X = filtered_data.drop(columns='VTR_PPP20')  # Features
y = filtered_data['VTR_PPP20']               # Target



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape



((25690, 184), (6423, 184))

In [6]:


# Identify categorical columns
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']

# Create transformers
# Impute and encode categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Impute numerical data
numerical_transformer = SimpleImputer(strategy='mean')

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, X_train.select_dtypes(exclude=['object']).columns),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Convert all categorical columns to string type
X_train[categorical_cols] = X_train[categorical_cols].astype(str)
X_test[categorical_cols] = X_test[categorical_cols].astype(str)




# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, X_train.select_dtypes(exclude=['object']).columns),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Preprocess training and testing data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)


# Initialize and train the GBM classifier
gbm_classifier = GradientBoostingClassifier(random_state=42)
gbm_classifier.fit(X_train_preprocessed, y_train)

# Predict on the test set
y_pred = gbm_classifier.predict(X_test_preprocessed)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(accuracy, classification_rep)



0.8013389381908765               precision    recall  f1-score   support

           D       0.84      0.84      0.84      3978
           R       0.74      0.73      0.74      2445

    accuracy                           0.80      6423
   macro avg       0.79      0.79      0.79      6423
weighted avg       0.80      0.80      0.80      6423



In [7]:
# Extract feature importances from the trained GBM classifier
feature_importances = gbm_classifier.feature_importances_

# Get feature names after preprocessing (to account for one-hot encoded features)
feature_names = (preprocessor.named_transformers_['cat']['encoder']
                 .get_feature_names_out(categorical_cols))

# Combine feature names for numerical and one-hot encoded categorical features
all_feature_names = list(X_train.select_dtypes(exclude=['object']).columns) + list(feature_names)

# Sort features based on importance
sorted_idx = feature_importances.argsort()[::-1]
top_20_features = [(all_feature_names[i], feature_importances[i]) for i in sorted_idx[:60]]

top_20_features


[('TOD_PRES_R_2020_PREC', 0.22858892040794523),
 ('TOD_PRES_D_2020_PREC', 0.13974694199869364),
 ('TOD_PRES_R_2016_PREC', 0.08817951453923158),
 ('TOD_PRES_R_2016', 0.08408141198603027),
 ('ETHNIC_INFER_C', 0.0838612038294447),
 ('STATE_VA', 0.04100074607521282),
 ('ETHNIC_INFER_B', 0.0405969728141887),
 ('CENSUS_ST_51', 0.03647938481932905),
 ('STATE_SC', 0.029132833668422654),
 ('CENSUS_ST_45', 0.023677148380847918),
 ('AGE', 0.020068973238698974),
 ('VP_PRI', 0.019864155224919102),
 ('VP_PPP', 0.016385858119801918),
 ('GENDER_MIX_4', 0.014306952151942515),
 ('TOD_PRES_O_2016', 0.012700088519986247),
 ('TOD_PRES_D_2016', 0.011808171816109504),
 ('CENSUS_ST_42', 0.01153646930729019),
 ('TOD_PRES_O_2016_PREC', 0.007495838518962777),
 ('CNSUS_PCTB', 0.006167734751774328),
 ('DON_POLCONS_ ', 0.005159930569222308),
 ('STATE_AL', 0.005067089767020803),
 ('RELIGINSP_Y', 0.004936223191070591),
 ('CENSUS_ST_13', 0.004569613821717729),
 ('DON_POLCONS_Y', 0.004040018175731152),
 ('SEX_M', 0.003

In [ ]:
PARTY_CODE
PARTY_MIX
PRFL_BIDEN_SUPPORT 
PRFL_BORDER_SECURITY
PRFL_CONSERVATIVE_NEWS 
PRFL_FENCE_SITTER 
PRFL_IMMIGRATION_REFORM 
PRFL_LGBT_SUPPORT
PRFL_LIBERAL_NEWS 
PRFL_MARIJUANA_REFORM
PRFL_MINWAGE 
PRFL_PERSUADABLE_VOTER
PRFL_SANDERS_SUPPORT
PRFL_TRUMP_SUPPORT 

VTR_GEN00	VTR_GEN01	VTR_GEN02	VTR_GEN03	VTR_GEN04	VTR_GEN05	VTR_GEN06	VTR_GEN07	VTR_GEN08	VTR_GEN09	VTR_GEN10	VTR_GEN11	VTR_GEN12	VTR_GEN13	VTR_GEN14	VTR_GEN15	VTR_GEN16	VTR_GEN17	VTR_GEN18	VTR_GEN19	VTR_GEN20	VTR_GEN21	VTR_GEN22	VTR_OTH00	VTR_OTH01	VTR_OTH02	VTR_OTH03	VTR_OTH04	VTR_OTH05	VTR_OTH06	VTR_OTH07	VTR_OTH08	VTR_OTH09	VTR_OTH10	VTR_OTH11	VTR_OTH12	VTR_OTH13	VTR_OTH14	VTR_OTH15	VTR_OTH16	VTR_OTH17	VTR_OTH18	VTR_OTH19	VTR_OTH20	VTR_OTH21	VTR_OTH22	VTR_PPP00	VTR_PPP04	VTR_PPP08	VTR_PPP12	VTR_PPP16	VTR_PPP20	VTR_PRI00	VTR_PRI01	VTR_PRI02	VTR_PRI03	VTR_PRI04	VTR_PRI05	VTR_PRI06	VTR_PRI07	VTR_PRI08	VTR_PRI09	VTR_PRI10	VTR_PRI11	VTR_PRI12	VTR_PRI13	VTR_PRI14	VTR_PRI15	VTR_PRI16	VTR_PRI17	VTR_PRI18	VTR_PRI19	VTR_PRI20	VTR_PRI21	VTR_PRI22

TOD_PRES_D_XXXX
TOD_PRES_O_XXXX
TOD_PRES_R_XXXX
TOD_PRES_DIFF_XXXX
TOD_SEN_DIFF_XXXX
TOD_PRES_D_2016
TOD_PRES_R_2016
TOD_PRES_DIFF_2016_PREC
TOD_PRES_D_2020
TOD_PRES_R_2020
TOD_PRES_DIFF_2020_PREC

### Model PARTY_CODE without any voter information